# 推論構造化出力のためのチェーン呼び出しの使用

最初にリリースされたバージョン（2024年9月）の[o1](https://openai.com/index/introducing-openai-o1-preview/)推論モデルは高度な機能を持っていますが、[構造化出力](https://platform.openai.com/docs/guides/structured-outputs/examples)をサポートしていません。

これは、o1を使用したリクエストでは信頼性のある型安全性がなく、有用なJSONを返すためにプロンプト自体に依存することを意味します。

このガイドでは、OpenAI APIを使用する際に、o1モデル、特に`o1-preview`に有効なJSON形式を返すようプロンプトする2つの方法を探ります。

# プロンプティング

`o1-preview`を使用してJSONレスポンスを返す最もシンプルな方法は、明示的にプロンプトすることです。

以下の例を見てみましょう：
- 企業のWikipediaページを取得する
- AI機能から最も恩恵を受けられる企業を判定する
- JSON形式で返す（他のシステムで取り込み可能な形式）

In [11]:
import requests
from openai import OpenAI

client = OpenAI()

def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

url = "https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue"
html_content = fetch_html(url)

json_format = """
{
    companies: [
        {
            \"company_name\": \"OpenAI\",
            \"page_link\": \"https://en.wikipedia.org/wiki/OpenAI\",
            \"reason\": \"OpenAI would benefit because they are an AI company...\"
        }
    ]
}
"""

o1_response = client.chat.completions.create(
    model="o1-preview",
    messages=[
        {
            "role": "user", 
            "content": f"""
You are a business analyst designed to understand how AI technology could be used across large corporations.

- Read the following html and return which companies would benefit from using AI technology: {html_content}.
- Rank these propects by opportunity by comparing them and show me the top 3. Return only as a JSON with the following format: {json_format}"
"""
        }
    ]
)

print(o1_response.choices[0].message.content)

{
    "companies": [
        {
            "company_name": "Walmart",
            "page_link": "https://en.wikipedia.org/wiki/Walmart",
            "reason": "Walmart could benefit from AI technology by enhancing their supply chain management, optimizing inventory levels, improving customer service through AI-powered chatbots, and providing personalized shopping experiences. AI can help Walmart forecast demand more accurately, reduce operational costs, and increase overall efficiency."
        },
        {
            "company_name": "UnitedHealth Group",
            "page_link": "https://en.wikipedia.org/wiki/UnitedHealth_Group",
            "reason": "UnitedHealth Group could leverage AI technology to improve patient care through predictive analytics, personalize treatment plans, detect fraudulent claims, and streamline administrative processes. AI can assist in early disease detection, improve diagnostic accuracy, and enhance data analysis for better health outcomes."
        },
   

応答は既にかなり良好であることに注意してください - 適切な応答を含むJSONを返しています。しかし、プロンプトのみのJSON推論の既存のユースケースと同じ落とし穴に陥ります：
- このJSONを型安全な構造に手動で処理する必要があります
- モデルの拒否は[APIから別の構造として明示的に返されません](https://platform.openai.com/docs/guides/structured-outputs/refusals)

# 構造化出力

それでは、[構造化出力](https://platform.openai.com/docs/guides/structured-outputs)を使ってこれを行ってみましょう。この機能を有効にするために、`o1-preview`のレスポンスを`gpt-4o-mini`への後続リクエストと連携させます。`gpt-4o-mini`は、最初の`o1-preview`レスポンスから返されたデータを効果的に処理することができます。

In [12]:
from pydantic import BaseModel
from devtools import pprint

class CompanyData(BaseModel):
    company_name: str
    page_link: str
    reason: str

class CompaniesData(BaseModel):
    companies: list[CompanyData]

o1_response = client.chat.completions.create(
    model="o1-preview",
    messages=[
        {
            "role": "user", 
            "content": f"""
You are a business analyst designed to understand how AI technology could be used across large corporations.

- Read the following html and return which companies would benefit from using AI technology: {html_content}.
- Rank these propects by opportunity by comparing them and show me the top 3. Return each with {CompanyData.__fields__.keys()}
"""
        }
    ]
)

o1_response_content = o1_response.choices[0].message.content

response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user", 
            "content": f"""
Given the following data, format it with the given response format: {o1_response_content}
"""
        }
    ],
    response_format=CompaniesData,
)

pprint(response.choices[0].message.parsed)

CompaniesData(
    companies=[
        CompanyData(
            company_name='Walmart',
            page_link='https://en.wikipedia.org/wiki/Walmart',
            reason=(
                'As the largest retailer, Walmart can significantly benefit from AI by optimizing supply chain and inv'
                'entory management, improving demand forecasting, personalizing customer experiences, and enhancing in'
                '-store operations through AI-driven analytics.'
            ),
        ),
        CompanyData(
            company_name='JPMorgan Chase',
            page_link='https://en.wikipedia.org/wiki/JPMorgan_Chase',
            reason=(
                'As a leading financial institution, JPMorgan Chase can leverage AI for fraud detection, risk manageme'
                'nt, personalized banking services, algorithmic trading, and enhancing customer service with AI-powere'
                'd chatbots and virtual assistants.'
            ),
        ),
        CompanyData(
  

# 結論

構造化出力により、コードに信頼性の高い型安全性とより簡潔なプロンプトを提供できます。さらに、オブジェクトスキーマを再利用することで、既存のワークフローへの統合が容易になります。

o1クラスのモデルは現在、構造化出力をサポートしていませんが、2つのリクエストを連鎖させることで、`gpt-4o-mini`の既存の構造化出力機能を再利用できます。このフローは現在2回の呼び出しが必要ですが、2回目の`gpt-4o-mini`呼び出しのコストは、`o1-preview`/`o1-mini`の呼び出しと比較して最小限に抑えられるはずです。